## Tarea 5

##  Punto 1

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

min_val, max_val, step = 0, 20, 0.1  

x = np.arange(min_val, max_val + step, step)  
y = np.arange(min_val, max_val + step, step)
h = step

def sup(y_val):
    return 10

def inf(y_val):
    return 0

def izq(x_val):
    return 0

def der(x_val):
    return 0

def iniciar_pot():
    pot = np.zeros((len(x), len(y)))
    pot[0, :] = sup(y)
    pot[-1, :] = inf(y)
    pot[:, 0] = izq(x)
    pot[:, -1] = der(x)
    return pot

def fuente(x_val, y_val):
    return np.exp(-x_val * y_val)

def poisson(pot, fuente, i=int(1e5), w=1.5, tolerancia=1e-3):
    for i in tqdm(range(i), desc='Iteraciones'):
        max_dif = 0
        for i in range(1, len(x)-1):
            for j in range(1, len(y)-1):
                valor_old = pot[i, j]
                valor_new = (pot[i+1, j] + pot[i-1, j] + pot[i, j+1] + pot[i, j-1])/4 - h**2 * fuente(x[i], y[j])/4
                pot[i, j] = valor_old + w * (valor_new - valor_old)
                max_dif = max(max_dif, abs(pot[i, j] - valor_old))
        if max_dif < tolerancia:
            break
            
    return pot

V = iniciar_pot()
V = poisson(V, fuente)

Iteraciones:   0%|          | 0/100000 [00:00<?, ?it/s]

## Punto 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from tqdm.notebook import tqdm

num_t = 100
num_x = 20
num_y = 20

x = np.linspace(0, 1, num_x)
y = np.linspace(0, 1, num_y)
t = np.linspace(0, 1, num_t)

dx = x[1] - x[0]
dy = y[1] - y[0]
dt = t[1] - t[0]

alpha = 0.05

lx = (alpha*dt)/(dx**2)
ly = (alpha*dt)/(dy**2)

def inicial(x, y):
    return np.sin(np.pi * (x + y))

def c_x0(y, t):
    return np.exp(-2*np.pi**2*t) * np.sin(np.pi*y)

def c_x1(y, t):
    return np.exp(-2*np.pi*2*t) * np.sin(np.pi(1 + y))

def c_y0(x, t):
    return np.exp(-2*np.pi**2*t) * np.sin(np.pi*x)

def c_y1(x, t):
    return np.exp(-2*np.pi*2*t) * np.sin(np.pi(1 + x))

def temp_inicial():
    temp = np.zeros((num_t, num_x, num_y))
    
    for i in range(num_x):
        for j in range(num_y):
            temp[0, i, j] = inicial(x[i], y[j])
    
    temp[:, 0, :] = c_x0(y, t)
    temp[:, -1, :] = c_x1(y, t)
    temp[:, :, 0] = c_y0(x, t)
    temp[:, :, -1] = c_y1(x, t)
    
    return temp

grid_temp = temp_inicial()

def resolver_ec():
    for step_t in tqdm(range(1, num_t)):
        for i in range(1, num_x-1):
            for j in range(1, num_y-1):
                grid_temp[step_t, i, j] = (
                    grid_temp[step_t-1, i, j]
                    + lx * (grid_temp[step_t-1, i+1, j] - 2*grid_temp[step_t-1, i, j] + grid_temp[step_t-1, i-1, j])
                    + ly * (grid_temp[step_t-1, i, j+1] - 2*grid_temp[step_t-1, i, j] + grid_temp[step_t-1, i, j-1])
                )

resolver_ec()

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111, projection='3d')

X, Y = np.meshgrid(x, y)

def init():
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_zlim(0, 10)
    return fig,

def grafico(frame):
    ax.clear()
    init()
    ax.plot_surface(X, Y, grid_temp[frame, :, :], cmap='viridis')
    return fig,

anim_calor = animation.FuncAnimation(fig, grafico, frames=num_t, init_func=init, blit=False)
plt.show()

## Punto 5

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

v = 2
lx = 2
ly = 2
nx = 20
ny = 20
nt = 400
tf = 3

x = np.linspace(0, lx, nx)
y = np.linspace(0, ly, ny)
t = np.linspace(0, tf, nt)
dx = x[1] - x[0]
dy = y[1] - y[0]
dt = t[1] - t[0]

lxm = (v * dt / dx) ** 2
lym = (v * dt / dy) ** 2

def initial_condition(x, y):
    return np.sin(np.pi * x) * np.sin(np.pi * y)

u = np.zeros((nt, nx, ny))
u[0,:,:] = initial_condition(*np.meshgrid(x, y))

for n in range(1, nt-1):
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            u[n+1, i, j] = (2 * (1 - lxm - lym) * u[n, i, j] +
                            lxm * (u[n, i+1, j] + u[n, i-1, j]) +
                            lym * (u[n, i, j+1] + u[n, i, j-1]) -
                            u[n-1, i, j])

def init():
    ax.set_xlim(0, lx)
    ax.set_ylim(0, ly)
    ax.set_zlim(-1, 1)
    return fig,

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(x, y)

ani = animation.FuncAnimation(fig, init_func=init, frames=nt, interval=20, blit=True)

def animate(frame):
    ax.cla()
    ax.plot_surface(X, Y, u[frame,:,:], cmap='viridis')
    ax.set_xlim(0, lx)
    ax.set_ylim(0, ly)
    ax.set_zlim(-1, 1)
    return fig,

HTML(ani.to_html5_video())